In [ ]:
import pandas as pd
import numpy as np
import random

# Load datasets
fertilizer_df = pd.read_csv('pro_fert_data.csv')
crop_df = pd.read_csv('pro_crop_data.csv')

# Helper function to calculate mode for columns with object type
def mode(series):
    return series.mode().iloc[0]

# Function to recommend fertilizers
def recommend_fertilizers(crop_name, growth_stage, water_quality):
    # Filter crop data based on crop_name and growth_stage
    crop_data = crop_df[(crop_df['crop_name'] == crop_name) & (crop_df['crop_growth_stage'] == growth_stage)]

    if len(crop_data) == 0:
        return "Crop information not found."

    # Calculate the average for numeric columns and mode for categorical columns
    crop_nutrients = crop_data.drop(['crop_id', 'crop_name', 'crop_growth_stage', 'crop_type'], axis=1).apply(
        lambda x: mode(x) if x.dtype == 'O' else x.mean(), axis=0)

    # Define nutrient columns
    nutrient_columns = ['fert_nitrogen', 'fert_phosphorus', 'fert_potassium', 'fert_calcium', 'fert_magnesium', 'fert_sulfur',
                        'fert_copper', 'fert_chlorine', 'fert_boron', 'fert_iron', 'fert_zinc', 'fert_manganese',
                        'fert_molybdenum', 'fert_nickel', 'fert_cobalt', 'fert_sodium']

    # Calculate nutrient deficits
    deficits = {}
    for nutrient in nutrient_columns:
        crop_requirement = crop_nutrients.get(nutrient, 0)
        water_level = water_quality.get(nutrient.split('_')[1], 0)
        deficit = max(0, crop_requirement - water_level)
        if deficit > 0:
            deficits[nutrient] = deficit

    # If no deficits, return no fertilizer needed
    if not deficits:
        return "No fertilizer needed. Water quality meets all nutrient requirements."

    # Handle missing values in fertilizer dataset
    fertilizer_df.fillna(0, inplace=True)

    # Find fertilizers to meet the deficits
    recommended_fertilizers = []
    for nutrient, deficit in deficits.items():
        fert_candidates = fertilizer_df[fertilizer_df[nutrient] > 0]
        fert_candidates['contribution'] = fert_candidates[nutrient]
        fert_candidates = fert_candidates.sort_values(by='contribution', ascending=False)

        total_contribution = 0
        fert_combination = []
        for _, fert in fert_candidates.iterrows():
            if total_contribution >= deficit:
                break
            fert_combination.append(fert)
            total_contribution += fert[nutrient]

        for fert in fert_combination:
            recommended_fertilizers.append({
                'fert_id': fert['fert_id'],
                'fert_name': fert['fert_name'],
                'fert_toxicity': fert['fert_toxicity'],
                'fert_solubility': fert['fert_solubility'],
                'fert_temp': fert['fert_temp'],
                'fert_role': fert['fert_role'],
                'fert_remark': fert['fert_remark'],
                'quantity_KG_per_1000L': random.randint(100, 40000) / 1000
            })

    # Convert to DataFrame
    recommended_fertilizers_df = pd.DataFrame(recommended_fertilizers)

    # Save to CSV
    recommended_fertilizers_df.to_csv('recommended_fertilizers.csv', index=False)

# Example usage:
crop_name_input = "baylaurel"
growth_stage_input = "vegetative"
water_quality_input = {
    'ph': 6.5,
    'temp': 25,
    'ec': 1.2,
    'nitrogen': 100,
    'phosphorus': 50,
    'potassium': 150,
    'calcium': 80,
    'magnesium': 30,
    'sulfur': 20,
    'copper': 0.5,
    'chlorine': 10,
    'boron': 0.3,
    'iron': 1.0,
    'zinc': 0.1,
    'manganese': 0.05,
    'molybdenum': 0.02,
    'nickel': 0.01,
    'cobalt': 0.01,
    'sodium': 5
}

recommend_fertilizers(crop_name_input, growth_stage_input, water_quality_input)

'No fertilizer needed. Water quality meets all nutrient requirements.'